In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path
from platform import system
from difflib import get_close_matches

import util
import lines
import group
import label
import records
import extract
import date

In [2]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir, suffix=".pdf")
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[2]

indexes_path = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/tesseract_data_frames/"
output_dir = "/home/jeusa/Files/Documents/Indexex/ILO_yearly_indices/extracted_indexes/"

if system()=="Windows":
    indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
    output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

indexes_csv = util.list_files(indexes_path, suffix=".csv")

display(test_dfs, file)

['.././test_files/tesseract_data_frames/LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/LS_index_1968.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986_split.csv',
 '.././test_files/tesseract_data_frames/LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/LS_index_1920.csv',
 '.././test_files/tesseract_data_frames/LS_index_1985-1.csv']

'.././test_files/tesseract_data_frames/LS_index_1968.csv'

In [3]:
pdf_file = test_files[-1]
display(test_files, pdf_file)

['.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1937.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1944_split.pdf',
 '.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/LS_index_1921.pdf',
 '.././test_files/LS_index_1985-1.pdf',
 '.././test_files/LS_index_1920_split.pdf',
 '.././test_files/LS_index_1968.pdf']

'.././test_files/LS_index_1968.pdf'

In [4]:
# fitz mode
start_page = 1

pdf_words, pdf_dicts = util.read_pdf(pdf_file, start_page)

w_df = lines.make_words_df(pdf_words, start_page)

w = w_df.rename(columns={"text": "line_text"})
w["x0"] = [round(x, 2) for x in w["x0"]]
w["y0"] = [round(x, 2) for x in w["y0"]]
w["x1"] = [round(x, 2) for x in w["x1"]]
w["y1"] = [round(x, 2) for x in w["y1"]]

w_l = lines.merge_close_lines(w)
w_l = lines.remove_useless_lines(w_l)

l_d = lines.make_lines_df_from_dicts(pdf_dicts, start_page)
l = lines.merge_close_lines(l_d)
l = lines.remove_useless_lines(l)

r_df = extract.extract_indexes(w_df, w_l, pdf_file, "fitz")
r_df

Reading pdf from .././test_files/LS_index_1968.pdf
...
Finished reading 83 page(s)


,country,region,text,date,year,page,extracted_date,full_text
0,BULGARIA,,"16 Dec. 1965 Decree No. 927, to amend and supp...",,,1,,"16 Dec. 1965 Decree No. 927, to amend and supp..."
1,CZECHOSLOVAKIA,,1 Dec. 1967 An Act to make certain further cha...,,,1,,1 Dec. 1967 An Act to make certain further cha...
2,EL SALVADOR,,"10 May 1954 Decree No. 37, to approve the Soci...",,,1,,"10 May 1954 Decree No. 37, to approve the Soci..."
3,EL SALVADOR,,"13 Mar. 1958 Decree No. 2607, to amend the Soc...",,,1,,"13 Mar. 1958 Decree No. 2607, to amend the Soc..."
4,NEW ZEALAND,,20 Oct. 1966 An Act to amend the Social Securi...,,,1,,20 Oct. 1966 An Act to amend the Social Securi...
...,...,...,...,...,...,...,...,...
1047,INTERNATIONAL,PortugalSpain,"3 June Administrative Agreement No. 2, on the ...",,,82,,"3 June Administrative Agreement No. 2, on the ..."
1048,INTERNATIONAL,PortugalSpain,1 L.S. 1950—Int. 4. — 92 —,,,82,,1 L.S. 1950—Int. 4. — 92 —
1049,INTERNATIONAL,PortugalSpain,INTERNATIONAL (cont.),,,83,,INTERNATIONAL (cont.)
1050,INTERNATIONAL,PortugalSpain,Sweden-Yugoslavia 5 July Social Insurance Conv...,,,83,,Sweden-Yugoslavia 5 July Social Insurance Conv...


In [152]:
# lax versions
digit = "[0-9oOIltriSQzZ]"
big_i = "[I1l]"
one = "[1Iltri]"

day = "(?<![0-9])([1-3IltrizZ]?" + digit + ")(?![0-9])" # 4 | 31
dayth = day + "(st|nd|rd|th)?" # 4th | 31st
month_short = "([A-Za-z]{3}[.:,]{0,3}|[A-Za-z]{4})" # Dec. | June
month_long = "([A-Za-z]{3,9})" # February
year = "(" + one + "9"  + digit + "{2})"

# in the beginning, example: Nov. 4 | July 25th
re_d1 = "^" + month_short + " " + dayth

# in the beginning, example: 13/III/1986 | 7/IX/1985
re_d2 = "^" + day + "[/,;.]{1,2}" + big_i + "{0,3}[VX]?" + big_i + "{0,3}[/,;.]{1,2}" + year

# towards the end, example: 25th February, 1929
re_d3 = dayth + " " + month_long + "[,.:]? " + year

# in the beginning, example: 16 Dec. 1965
re_d4 = "^" + dayth + " " + month_short + "[,.:]?( " + year + ")?"

In [164]:
# strict versions
digit = "[0-9]"
big_i = "[I1l]"
one = "[1Il]"

day = "(?<![0-9])([1-3]?" + digit + ")(?![0-9])" # 4 | 31
dayth = day + "(st|nd|rd|th)?" # 4th | 31st
month_short = "([A-Za-z]{3}\.{0,3}|[A-Za-z]{4})" # Dec. | June
month_long = "([A-Za-z]{3,9})" # February
year = "(" + one + "9"  + digit + "{2})"

# in the beginning, example: Nov. 4 | July 25th
re_d1 = "^" + month_short + " " + dayth

# in the beginning, example: 13/III/1986 | 7/IX/1985
re_d2 = "^" + day + "/" + big_i + "{0,3}[VX]?" + big_i + "{0,3}/" + year

# towards the end, example: 25th February, 1929
re_d3 = dayth + " " + month_long + "[,.:]? " + year

# in the beginning, example: 16 Dec. 1965 | 7 May
re_d4 = "^" + dayth + " " + month_short + "[,.:]?( " + year + ")?"

In [167]:
re.search(re_d4, "16 Dec. 1998 Hello")

<re.Match object; span=(0, 12), match='16 Dec. 1998'>

In [37]:
def get_date_type(rec_df):

    # stricter version of the date regex's
    digits = "[0-9]"
    re_d1 = "^(([A-Za-z]{3}[.:,]{0,3}|[A-Za-z]{4}.?) [1-3]?" + digits + "(?![0-9])(st|nd|rd|th)?)" # in the beginning, example: Nov. 4 | July 25th
    re_d2 = "^([1-3]?" + digits + "/[I1l]{0,3}[VX]?[I1l]{0,3}/" + digits + "{4})" # in the beginning, example: 13/III/1986 | 7/11/198S
    re_d3 = "([1-3]?" + digits + "(st|nd|rd|th)? [A-Za-z]{3,}[,.] ?[I1l]" + digits + "{3})" # towards the end, example: 25th February, 1929
    re_d4 = "^" + re_d3 # in the beginning, example: 16 Dec. 1965

    samp = rec_df.sample(frac=1/10)
    samp["date_type"] = -1

    for i, row in samp.iterrows():
        t = row["text"]
        dt = -1

        d1 = re.search(re_d1, t)
        d2 = re.search(re_d2, t)
        d3 = None
        d4 = re.search(re_d4, t)

        for m in re.finditer(re_d3, t):
            d3 = m

            dt = 3
        if d2 != None:
            dt = 2
        if d1 != None:
            dt = 1
        if d4 != None:
            #print(d4.group())
            dt = 4

        samp.loc[i, ["date_type"]] = dt

    date_type = samp.groupby("date_type").count().sort_values("country", ascending=False)
    display(date_type)

    if not date_type.empty:
        date_type = date_type.iloc[0].name
    else:
        date_type = -1

    return date_type

In [38]:
get_date_type(r_df)

,country,region,text,date,year,page,extracted_date,full_text
date_type,,,,,,,,
-1,81,81,81,81,81,81,81,81
3,22,22,22,22,22,22,22,22
4,2,2,2,2,2,2,2,2


-1

In [9]:
b_x0, b_x1, c0_c = group.group_line_starts_ends(l, "fitz")
bor = lines.make_borders_df(b_x0, b_x1)

In [10]:
# tess mode
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df, "tess")
borders = lines.make_borders_df(bins_x0, bins_x1)

rec_df = extract.extract_indexes(pdf_df, lines_df, file, "tess")
rec_df

,country,region,text,date,year,page,extracted_date,full_text
0,CZECHOSLOVAKIA,,1 Dec. 1967. An Act to make certain further ch...,,,1,,1 Dec. 1967. An Act to make certain further ch...
1,EL SALVADOR,,"10 May 1954 Decree No. 37, to approve the Soci...",,,1,,"10 May 1954 Decree No. 37, to approve the Soci..."
2,EL SALVADOR,,"13 Mar. 1958 Decree No. 2607, to amend the Soc...",,,1,,"13 Mar. 1958 Decree No. 2607, to amend the Soc..."
3,NEW ZEALAND,,20 Oct. 1966 An Act to amend the Social Securi...,,,1,,20 Oct. 1966 An Act to amend the Social Securi...
4,NEW ZEALAND,,7 June 1967 An Act to amend the Social Securit...,,,1,,7 June 1967 An Act to amend the Social Securit...
...,...,...,...,...,...,...,...,...
1024,INTERNATIONAL,PortugalSpain,"3 June Administrative Agreement No. 2, on the ...",,,82,,"3 June Administrative Agreement No. 2, on the ..."
1025,INTERNATIONAL,PortugalSpain,1L.S. 1950—Int. 4. 9?2 —,,,82,,1L.S. 1950—Int. 4. 9?2 —
1026,INTERNATIONAL,Sweden Yugoslavia,5 July Social Insurance Convention. (Sweden: S...,,,83,,5 July Social Insurance Convention. (Sweden: S...
1027,INTERNATIONAL,SwitzerlandUnited Kingdom,21 Feb. Convention on social security [repeals...,,,83,,21 Feb. Convention on social security [repeals...


## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung FIXED
- Date-Type-Erkennung fehlerhaft
    - 1920 FIXED
- 1920 Seite 11 Uruguay
    - unvollständige Datums-Erkennung FIXED
    

## Todo
- doppelseitige Extraktion funktioniert auch im fitz-Modus
- Kommandozeilenprogramm